In [2]:
from utils import *
from extraction import *
import tensorflow as tf
from matplotlib.colors import ListedColormap

2024-03-17 10:56:21.250001: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 10:56:21.250094: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 10:56:21.251018: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-17 10:56:21.256957: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-17 10:56:21.930575: W tensorflow/compiler/tf2

In [3]:
if not os.path.exists(MASK_TESTING_DIR):
    os.makedirs(MASK_TESTING_DIR)
if not os.path.exists(MASK_TRAINING_DIR):
    os.makedirs(MASK_TRAINING_DIR)

In [ ]:
model = tf.keras.models.load_model(MODEL_PATH)

In [ ]:
def standardise(array_4d):
    num_classes = array_4d.shape[3]
    quantized_data = np.empty(array_4d.shape, dtype=np.int8)
    for class_idx in range(num_classes):
        class_data = array_4d[:, :, :, class_idx]
        class_data = np.round(class_data)
        quantized_data[:, :, :, class_idx] = class_data.astype(np.int8)
    return np.argmax(quantized_data, axis=-1)

In [ ]:
for i in range(1, PT_NUM+1):
    ptnum = str(i).zfill(3)
    save_path = os.path.join(MASK_TRAINING_DIR, f"patient{ptnum}_gt.nii")
    if os.path.exists(save_path):
        continue;
    img = nib_from_int(i)
    total_frames = training_data_DF.loc[i-1,'NbFrame']
    all_frames = []
    for j in range(int(total_frames)):
        frame = img.get_fdata()[:,:,:,j]
        frame = img4d_extraction(nib.Nifti1Image(frame, img.affine), CROP_SIZE)
        frame = np.expand_dims(frame, axis=0)
        img_mask = model.predict(frame)
        img_mask = img_mask[0,:,:,:,:]
        img_mask = standardise(img_mask)
        all_frames.append(img_mask)
    all_frames = np.transpose(np.asarray(all_frames), axes=(1, 2, 3, 0))
    affine = np.diag([1.25, 1.25, 10.0, 1.0])
    nifti_mask = nib.Nifti1Image(all_frames, affine=affine, dtype=np.int8)
    nib.save(nifti_mask, save_path)

In [ ]:
mask_4d = load_mask(1).get_fdata()
plt.figure(figsize = (10,10))
plt.imshow(mask_4d[:,:,5,5], cmap = 'gray')
plt.title("Image Data")
plt.colorbar()
plt.show()

In [ ]:
plot_gen_mask(5,8)

In [ ]:
for i in range(PT_NUM+1, PT_NUM+TEST_PT_NUM+1):
    ptnum = str(i).zfill(3)
    save_path = os.path.join(MASK_TESTING_DIR, f"patient{ptnum}_gt.nii")
    if os.path.exists(save_path):
        continue;
    img = nib_from_int(i,testing=True)
    total_frames = testing_data_DF.loc[i-PT_NUM-1,'NbFrame']
    all_frames = []
    for j in range(int(total_frames)):
        frame = img.get_fdata()[:,:,:,j]
        frame = img4d_extraction(nib.Nifti1Image(frame, img.affine), CROP_SIZE)
        frame = np.expand_dims(frame, axis=0)
        img_mask = model.predict(frame)
        img_mask = img_mask[0,:,:,:,:]
        img_mask = standardise(img_mask)
        all_frames.append(img_mask)
    all_frames = np.transpose(np.asarray(all_frames), axes=(1, 2, 3, 0))
    affine = np.diag([1.25, 1.25, 10.0, 1.0])
    nifti_mask = nib.Nifti1Image(all_frames, affine=affine, dtype=np.int8)
    nib.save(nifti_mask, save_path)